# Notebook Setup

In [1]:
if 'google.colab' in str(get_ipython()):
  IN_COLLAB = True
else:
  IN_COLLAB = False

#TODO: CHANGE THIS BASED ON YOUR OWN LOCAL SETTINGS
MY_HOME_ABS_PATH = "/content/drive/MyDrive/W210/co2-flux-hourly-gpp-modeling"

In [2]:
if IN_COLLAB:
  from google.colab import drive
  drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Import Modules

In [3]:
# install required modules quietly
required_packages = ['geopandas','azure-storage-blob']

for p in required_packages: 
  try:
      __import__(p)
  except ImportError:
      %pip install {p} --quiet

In [4]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import math
import json

import pandas as pd
from calendar import monthrange
from datetime import datetime
from io import BytesIO

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Load locale custome modules
import sys
if IN_COLLAB:
  os.chdir(MY_HOME_ABS_PATH)
  sys.path.insert(0,os.path.abspath("./code/src/tools"))
else:
  sys.path.append(os.path.abspath("./code/src/tools"))

from CloudIO.AzStorageClient import AzStorageClient
from data_pipeline_lib import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

## Define Local File System Constants

In [5]:
root_dir =  MY_HOME_ABS_PATH
tmp_dir =  root_dir + os.sep + '.tmp'
raw_data_dir = tmp_dir
data_dir = root_dir + os.sep + 'data'
cred_dir = root_dir + os.sep + '.cred'
az_cred_file = cred_dir + os.sep + 'azblobcred.json'

if IN_COLLAB:
  raw_data_dir = "/content/drive/MyDrive/CO2_flux_gpp_modeling/DS_capstone_23Spring_CO2/Data/half_hourly_data"

In [6]:
train_sites = ['US-NR1', 'US-Me2', 'US-ARM', 'US-Vcp']
valid_sites = ["US-Vcp"]
test_sites = ["US-GLE"]
selected_sites =  train_sites + valid_sites + test_sites

# Load raw dataset from Azure Storage Blob

In [7]:
# Define target dataset
container = "baseline-data"
ext = "parquet"
ver = "0"
tag = "mc_raw"
blob_name_base = f"baseline_all_v_{ver}"
data_blob_name = f"{blob_name_base}_{tag}.{ext}"

full_blob_name = 'baseline_all_v_1-i-knn_raw.parquet'
train_blob_name = "baseline-train-v-1-i-knn.parquet"
test_blob_name = "baseline-test-v-1-i-knn.parquet"

tft_data_dir = tmp_dir + os.sep + 'data' + os.sep + 'gpp'

download_data = False

In [8]:
def load_data(container, data_blob_name, local_file_path):
  if not (os.path.exists(local_file_path)):
    if not (os.path.exists(tmp_dir)):
      os.mkdir(tmp_dir)
    if not (os.path.exists(tmp_dir + os.sep + 'data')):
      os.mkdir(tmp_dir + os.sep + 'data')
      os.mkdir(tmp_dir + os.sep + 'data' + os.sep + 'gpp')
    print(f"Loading dataset from {container}/{data_blob_name}...")
    # Initialized Azure Storage Client
    azStorageClient = AzStorageClient(az_cred_file)
    file_stream = azStorageClient.downloadBlob2Stream(container, data_blob_name)
    data_df = pd.read_parquet(file_stream, engine='pyarrow')
    data_df.to_csv(local_file_path)
  else:
    print(f"Loading dataset from {local_file_path}...")
    data_df = pd.read_csv(local_file_path)

  return data_df

In [9]:
if download_data:
  # Load full dataset
  full_local_file = tft_data_dir + os.sep + 'full.csv'

  full_df = load_data(container, full_blob_name, full_local_file)

  print(f"Data size: {full_df.shape}")
  print(full_df['site_id'].unique())
  print("data peak:")
  display(full_df.head())

In [10]:
if download_data:
  # Load train dataset
  train_local_file = tft_data_dir + os.sep + 'train.csv'

  if not (os.path.exists(train_local_file)):
    train_df = load_data(container, train_blob_name, tmp_dir + os.sep + 'train_raw.csv')
    print(f"Train taw size: {train_df.shape}")
    print(train_df['site_id'].unique())

    train_df = train_df[train_df['site_id'].isin(train_sites)]
    train_df.to_csv(train_local_file)
  else:
    train_df = pd.read_csv(train_local_file)

  print(f"Data size: {train_df.shape}")
  print(train_df['site_id'].unique())
  print("Train data peak:")
  display(train_df.head())

In [11]:
if download_data:
  # Load val and test dataset
  val_local_file = tft_data_dir + os.sep + 'validation.csv'
  test_local_file = tft_data_dir + os.sep + 'test.csv'

  if (not (os.path.exists(val_local_file))) or (not (os.path.exists(test_local_file))):
    data_df = load_data(container, test_blob_name, tmp_dir + os.sep + "test_raw.csv")
    print(f"Test Raw data size: {data_df.shape}")
    print(data_df['site_id'].unique())

    validation_df = data_df[data_df['site_id'].isin(valid_sites)]
    validation_df.to_csv(val_local_file)

    test_df = data_df[data_df['site_id'].isin(test_sites)]
    test_df.to_csv(test_local_file)
  else:
    validation_df = pd.read_csv(val_local_file)
    test_df = pd.read_csv(test_local_file)

  print(f"Validation data size: {validation_df.shape}")
  print(validation_df['site_id'].unique())
  print("Validation data peak:")
  display(validation_df.head())

  print(f"Test data size: {test_df.shape}")
  print(test_df['site_id'].unique())
  print("Test data peak:")
  display(test_df.head())

In [12]:
# import gc
# del train_df
# del validation_df
# del test_df
# gc.collect()

# Run TensortFlow TFT (Fix-Param, Test)

https://www.tensorflow.org/guide/migrate/upgrade

In [13]:
os.chdir(MY_HOME_ABS_PATH + "/code/src/tools/tft/")

In [14]:
!pip install tensorflow
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
import tensorflow as tf
print(tf.__version__)

2.11.0


In [16]:
# Update tesnort flow libs
# !tf_upgrade_v2 \
#     --intree libs/ \
#     --outtree libs_v2/ \
#     --reportfile tree_report.txt

In [17]:
tf.compat.v1.experimental.output_all_intermediates(True)

In [18]:
%run script_train_fixed_params.py gpp ../../../../.tmp no yes

Using output folder ../../../../.tmp


*** Training from defined parameters for gpp ***
Loading & splitting data...
Formatting train-valid-test splits.
Raw size: (865920, 50) from ['CA-Cbo' 'CH-Lae' 'ES-LJu' 'ES-LM2' 'FI-Hyy' 'FR-Lam' 'FR-Pue' 'IT-Lav'
 'US-AR1' 'US-ARM' 'US-GLE' 'US-NR1' 'US-SRM' 'US-Seg' 'US-Ton' 'US-UMB'
 'US-Var' 'US-Vcp' 'US-Wkg']
Setting scalers with training data...
IDs:['CA-Cbo', 'CH-Lae', 'ES-LJu', 'ES-LM2', 'FI-Hyy', 'FR-Lam', 'FR-Pue', 'IT-Lav', 'US-AR1', 'US-ARM', 'US-NR1', 'US-SRM', 'US-Seg', 'US-Ton', 'US-UMB', 'US-Var', 'US-Wkg']
Real number input: ['TA_ERA', 'SW_IN_ERA', 'LW_IN_ERA', 'VPD_ERA', 'P_ERA', 'PA_ERA', 'day', 'hour', 'lat', 'long', 'TA_ERA', 'SW_IN_ERA', 'LW_IN_ERA', 'VPD_ERA', 'P_ERA', 'PA_ERA', 'day', 'hour', 'lat', 'long'].
Setting scalers with training data...
Categorical input: ['year', 'month', 'IGBP', 'koppen_main', 'koppen_sub']
Train size: (96408, 50) from ['IT-Lav' 'US-NR1']
Valid size: (52584, 50) from ['US-Vcp']
Test size: (43824, 50) from ['US-GLE']
*** Loading hyperp

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


                                                                                                  
 activation_6 (Activation)      (None, 1, 5)         0           ['dense_42[0][0]']               
                                                                                                  
 activation_7 (Activation)      (None, 1, 5)         0           ['dense_46[0][0]']               
                                                                                                  
 activation_8 (Activation)      (None, 1, 5)         0           ['dense_50[0][0]']               
                                                                                                  
 dropout (Dropout)              (None, 5)            0           ['dense_23[0][0]']               
                                                                                                  
 dense_27 (Dense)               (None, 1, 5)         30          ['activation_2[0][0]']           
          

/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2333: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


Streaming output truncated to the last 5000 lines.
# input_size = 26
# output_size = 1
# category_counts = [6, 12, 8, 3, 7]
# input_obs_loc = [0]
# static_input_loc = [9, 10, 21, 22, 23, 24, 25]
# known_regular_inputs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
# known_categorical_inputs = [0, 1, 2, 3, 4]
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 192, 26)]    0           []                               
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['input_1[0][0]']                
 sformer/strided_slice_1 (Tenso                                                                   
 rFlowOpLayer)                                        

/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Training completed @ 2023-02-25 23:09:04.192183
Best validation loss = 5.2748875460803895
Params:
dropout_rate  =  0.1
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  100.0
minibatch_size  =  128
model_folder  =  ../../../../.tmp/saved_models/gpp/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('site_id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('datetime', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('GPP_NT_VUT_REF', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('TA_ERA', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('SW_IN_ERA', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('LW_IN_ERA', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('VPD_ERA', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('P_ERA', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), 

# Model Evaulation Visulation